# Premier League Data Collection for RAG Training

This notebook downloads Premier League history articles from Wikipedia and other sources to create a comprehensive dataset for RAG (Retrieval-Augmented Generation) training.

## 1. Install Required Libraries

Install necessary libraries for web scraping and Wikipedia API access.

In [1]:
!pip install requests beautifulsoup4 wikipedia-api lxml tqdm

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ----- ---------------------------------- 0.5/4.0 MB 4.2 MB/s eta 0:00:01
   ------------ --------------------------- 1.3/4.0 MB 4.2 MB/s eta 0:00:01
   -------------------- ------------------- 2.1/4.0 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 3.1/4.0 MB 3.9 MB/s eta 0:00:01
   -------------------------------------- - 3.9/4.0 MB 4.0 MB/s eta 0:00:01
   ---------------------------------------- 4.0/4.0 MB 3.9 MB/s  0:00:01
  Created wheel for wikipedia-api: filename=wikipedia_api-0.8.1-py3-none-any.whl size=15542 sha256=df6fbfe761bab320bbbbfe71353c6fe49fe0a22dcd81e9aa8

## 2. Import Dependencies

Import all necessary libraries for the data collection process.

In [2]:
import os
import json
import time
import requests
from bs4 import BeautifulSoup
import wikipediaapi
from datetime import datetime
from tqdm import tqdm
import re

## 3. Set Up Directory Structure

Create organized subdirectories for storing articles by source.

In [3]:
# Set up directory structure
base_dir = "premier_league_data"
subdirs = ["wikipedia", "metadata"]

# Create directories if they don't exist
os.makedirs(base_dir, exist_ok=True)
for subdir in subdirs:
    os.makedirs(os.path.join(base_dir, subdir), exist_ok=True)

print(f"✓ Directory structure created in '{base_dir}/'")
print(f"  - {base_dir}/wikipedia/")
print(f"  - {base_dir}/metadata/")

✓ Directory structure created in 'premier_league_data/'
  - premier_league_data/wikipedia/
  - premier_league_data/metadata/


## 4. Download Wikipedia Articles

Download comprehensive articles about Premier League history, teams, players, and seasons from Wikipedia.

In [4]:
# Initialize Wikipedia API with user agent
wiki_wiki = wikipediaapi.Wikipedia(
    language='en',
    user_agent='BizMind RAG Training Bot/1.0 (Educational Purpose)'
)

# Comprehensive list of Premier League related Wikipedia articles
wikipedia_articles = [
    # General Premier League
    "Premier League",
    "History of the Premier League",
    "Premier League records and statistics",
    "List of Premier League seasons",
    "Premier League Golden Boot",
    "Premier League Player of the Season",
    
    # Historic Seasons
    "1992–93 FA Premier League",
    "1993–94 FA Premier League",
    "1994–95 FA Premier League",
    "1995–96 FA Premier League",
    "1996–97 FA Premier League",
    "1997–98 FA Premier League",
    "1998–99 FA Premier League",
    "1999–2000 FA Premier League",
    "2000–01 FA Premier League",
    "2001–02 FA Premier League",
    "2002–03 FA Premier League",
    "2003–04 FA Premier League",
    "2004–05 FA Premier League",
    "2005–06 Premier League",
    "2006–07 Premier League",
    "2007–08 Premier League",
    "2008–09 Premier League",
    "2009–10 Premier League",
    "2010–11 Premier League",
    "2011–12 Premier League",
    "2012–13 Premier League",
    "2013–14 Premier League",
    "2014–15 Premier League",
    "2015–16 Premier League",
    "2016–17 Premier League",
    "2017–18 Premier League",
    "2018–19 Premier League",
    "2019–20 Premier League",
    "2020–21 Premier League",
    "2021–22 Premier League",
    "2022–23 Premier League",
    "2023–24 Premier League",
    "2024–25 Premier League",
    
    # Big Six Clubs
    "Manchester United F.C.",
    "Liverpool F.C.",
    "Arsenal F.C.",
    "Chelsea F.C.",
    "Manchester City F.C.",
    "Tottenham Hotspur F.C.",
    
    # Other Notable Clubs
    "Leicester City F.C.",
    "Everton F.C.",
    "Newcastle United F.C.",
    "Aston Villa F.C.",
    "West Ham United F.C.",
    
    # Legendary Players
    "Thierry Henry",
    "Alan Shearer",
    "Ryan Giggs",
    "Frank Lampard",
    "Steven Gerrard",
    "Wayne Rooney",
    "Cristiano Ronaldo",
    "Sergio Agüero",
    "Mohamed Salah",
    "Kevin De Bruyne",
    "Eric Cantona",
    "Dennis Bergkamp",
    "Didier Drogba",
    "Patrick Vieira",
    "Roy Keane",
    
    # Legendary Managers
    "Alex Ferguson",
    "Arsène Wenger",
    "José Mourinho",
    "Pep Guardiola",
    "Jürgen Klopp",
    
    # Notable Events
    "Treble (association football)",
    "Invincibles (English football)",
    "2015–16 Leicester City F.C. season",
]

print(f"Preparing to download {len(wikipedia_articles)} Wikipedia articles...")

Preparing to download 73 Wikipedia articles...


In [5]:
# Download Wikipedia articles
downloaded_articles = []
failed_articles = []

for article_title in tqdm(wikipedia_articles, desc="Downloading Wikipedia articles"):
    try:
        # Fetch the article
        page = wiki_wiki.page(article_title)
        
        if page.exists():
            # Clean the filename
            filename = re.sub(r'[^\w\s-]', '', article_title).strip().replace(' ', '_')
            filename = f"{filename}.txt"
            filepath = os.path.join(base_dir, "wikipedia", filename)
            
            # Save the article content
            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(f"Title: {page.title}\n")
                f.write(f"URL: {page.fullurl}\n")
                f.write(f"Downloaded: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                f.write("="*80 + "\n\n")
                f.write(page.text)
            
            downloaded_articles.append({
                "title": page.title,
                "filename": filename,
                "url": page.fullurl,
                "word_count": len(page.text.split()),
                "downloaded_at": datetime.now().isoformat()
            })
            
        else:
            failed_articles.append(article_title)
            print(f"\n⚠ Article not found: {article_title}")
        
        # Be respectful to Wikipedia servers
        time.sleep(0.5)
        
    except Exception as e:
        failed_articles.append(article_title)
        print(f"\n✗ Error downloading '{article_title}': {str(e)}")

print(f"\n✓ Successfully downloaded {len(downloaded_articles)} articles")
print(f"✗ Failed to download {len(failed_articles)} articles")


⚠ Article not found: Invincibles (English football)



✓ Successfully downloaded 72 articles
✗ Failed to download 1 articles


## 5. Scrape Premier League Official Website

Scrape historical content from the official Premier League website (if available).

In [6]:
# Note: This is a basic scraper example. The actual implementation would need
# to be adapted based on the current structure of premierleague.com

def scrape_premier_league_history():
    """
    Scrape Premier League history pages
    Note: This function needs to be adapted based on the current website structure
    """
    # Base URL
    base_url = "https://www.premierleague.com"
    
    # Headers to mimic browser
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    # Example: Scraping history page (this URL might need updating)
    urls_to_scrape = [
        f"{base_url}/history",
    ]
    
    scraped_content = []
    
    for url in urls_to_scrape:
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Extract main content (this selector needs to match actual site structure)
            # This is a generic example
            main_content = soup.find('main') or soup.find('article') or soup.find('body')
            
            if main_content:
                text = main_content.get_text(separator='\n', strip=True)
                
                # Save content
                filename = "premier_league_history.txt"
                filepath = os.path.join(base_dir, "wikipedia", filename)
                
                with open(filepath, 'w', encoding='utf-8') as f:
                    f.write(f"Source: {url}\n")
                    f.write(f"Downloaded: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
                    f.write("="*80 + "\n\n")
                    f.write(text)
                
                scraped_content.append({
                    "source": "Premier League Official",
                    "url": url,
                    "filename": filename
                })
                
            time.sleep(2)  # Respectful rate limiting
            
        except Exception as e:
            print(f"Error scraping {url}: {str(e)}")
    
    return scraped_content

# Uncomment to run (requires proper website structure analysis first)
# official_content = scrape_premier_league_history()
print("Note: Official website scraping function defined. Requires adaptation to current site structure.")

Note: Official website scraping function defined. Requires adaptation to current site structure.


## 6. Scrape Additional Sports News Sites

Collect articles from sports news websites (with rate limiting and respect for robots.txt).

In [7]:
# Example function for scraping news articles
# Note: Always check robots.txt and terms of service before scraping

def scrape_bbc_sport_pl_history():
    """
    Example function to scrape BBC Sport Premier League history
    This is for educational purposes and would need proper implementation
    """
    url = "https://www.bbc.com/sport/football/premier-league"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # This is a placeholder - actual implementation needs proper selectors
        print(f"✓ Successfully connected to {url}")
        print("Note: Full implementation requires proper HTML structure analysis")
        
    except Exception as e:
        print(f"Error: {str(e)}")

# Uncomment to run (requires proper implementation)
# scrape_bbc_sport_pl_history()
print("Note: Additional scraping functions defined but require proper implementation.")

Note: Additional scraping functions defined but require proper implementation.


## 7. Clean and Format Article Text

Process the scraped content to ensure consistent formatting.

In [8]:
def clean_text(text):
    """Clean and normalize text content"""
    # Remove excessive whitespace
    text = re.sub(r'\n\s*\n\s*\n', '\n\n', text)
    # Remove excessive spaces
    text = re.sub(r' +', ' ', text)
    # Remove special characters that might cause encoding issues
    text = text.encode('utf-8', errors='ignore').decode('utf-8')
    return text.strip()

def format_article(filepath):
    """Format and clean an article file"""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            content = f.read()
        
        cleaned_content = clean_text(content)
        
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(cleaned_content)
        
        return True
    except Exception as e:
        print(f"Error formatting {filepath}: {str(e)}")
        return False

# Clean all downloaded articles
wikipedia_dir = os.path.join(base_dir, "wikipedia")
article_files = [f for f in os.listdir(wikipedia_dir) if f.endswith('.txt')]

cleaned_count = 0
for article_file in tqdm(article_files, desc="Cleaning articles"):
    filepath = os.path.join(wikipedia_dir, article_file)
    if format_article(filepath):
        cleaned_count += 1

print(f"\n✓ Cleaned {cleaned_count} articles")

Cleaning articles: 100%|██████████| 72/72 [00:00<00:00, 598.75it/s]


✓ Cleaned 72 articles


## 8. Save Articles to Local Files

All articles have been saved during the download process with proper naming conventions.

In [9]:
# Verify all saved files
wikipedia_dir = os.path.join(base_dir, "wikipedia")
saved_files = [f for f in os.listdir(wikipedia_dir) if f.endswith('.txt')]

print(f"✓ Total articles saved: {len(saved_files)}")
print(f"✓ Location: {os.path.abspath(wikipedia_dir)}")

# Show sample filenames
print("\nSample filenames:")
for filename in saved_files[:5]:
    filepath = os.path.join(wikipedia_dir, filename)
    file_size = os.path.getsize(filepath) / 1024  # Size in KB
    print(f"  - {filename} ({file_size:.1f} KB)")

✓ Total articles saved: 72
✓ Location: d:\My Projects\BizMind\premier_league_data\wikipedia

Sample filenames:
  - 199293_FA_Premier_League.txt (10.9 KB)
  - 199394_FA_Premier_League.txt (6.3 KB)
  - 199495_FA_Premier_League.txt (6.8 KB)
  - 199596_FA_Premier_League.txt (5.8 KB)
  - 199697_FA_Premier_League.txt (3.8 KB)


## 9. Create Metadata Index

Generate a comprehensive index file with metadata for all downloaded articles.

In [10]:
# Save metadata as JSON
metadata_file = os.path.join(base_dir, "metadata", "articles_index.json")

metadata_export = {
    "collection_date": datetime.now().isoformat(),
    "total_articles": len(downloaded_articles),
    "articles": downloaded_articles,
    "failed_articles": failed_articles
}

with open(metadata_file, 'w', encoding='utf-8') as f:
    json.dump(metadata_export, f, indent=2, ensure_ascii=False)

print(f"✓ Metadata saved to {metadata_file}")

# Also save as a simpler CSV format for easy viewing
import csv

csv_file = os.path.join(base_dir, "metadata", "articles_index.csv")

with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    if downloaded_articles:
        writer = csv.DictWriter(f, fieldnames=downloaded_articles[0].keys())
        writer.writeheader()
        writer.writerows(downloaded_articles)

print(f"✓ CSV index saved to {csv_file}")

✓ Metadata saved to premier_league_data\metadata\articles_index.json
✓ CSV index saved to premier_league_data\metadata\articles_index.csv


## 10. Verify Downloaded Data

Create a comprehensive summary report of all collected data.

In [11]:
# Generate comprehensive summary report
total_words = sum(article['word_count'] for article in downloaded_articles)
total_size = sum(
    os.path.getsize(os.path.join(base_dir, "wikipedia", article['filename']))
    for article in downloaded_articles
) / (1024 * 1024)  # Convert to MB

print("="*80)
print("PREMIER LEAGUE DATA COLLECTION SUMMARY")
print("="*80)
print(f"\n📊 Collection Statistics:")
print(f"  • Total articles downloaded: {len(downloaded_articles)}")
print(f"  • Failed downloads: {len(failed_articles)}")
print(f"  • Total word count: {total_words:,}")
print(f"  • Total data size: {total_size:.2f} MB")
print(f"  • Average words per article: {total_words // len(downloaded_articles) if downloaded_articles else 0:,}")

print(f"\n📁 Data Location:")
print(f"  • Articles: {os.path.abspath(os.path.join(base_dir, 'wikipedia'))}")
print(f"  • Metadata: {os.path.abspath(os.path.join(base_dir, 'metadata'))}")

print(f"\n📝 Content Categories:")
categories = {
    "General PL": 0,
    "Seasons": 0,
    "Clubs": 0,
    "Players": 0,
    "Managers": 0,
    "Other": 0
}

for article in downloaded_articles:
    title = article['title'].lower()
    if any(x in title for x in ['premier league', 'history', 'records', 'golden boot']):
        categories["General PL"] += 1
    elif any(x in title for x in ['season', '–']):
        categories["Seasons"] += 1
    elif any(x in title for x in ['f.c.', 'united', 'city', 'villa']):
        categories["Clubs"] += 1
    elif any(x in title for x in ['henry', 'shearer', 'giggs', 'lampard', 'rooney', 'salah', 'agüero']):
        categories["Players"] += 1
    elif any(x in title for x in ['ferguson', 'wenger', 'mourinho', 'guardiola', 'klopp']):
        categories["Managers"] += 1
    else:
        categories["Other"] += 1

for category, count in categories.items():
    if count > 0:
        print(f"  • {category}: {count} articles")

print("\n✓ Data collection complete and ready for RAG training!")
print("="*80)

PREMIER LEAGUE DATA COLLECTION SUMMARY

📊 Collection Statistics:
  • Total articles downloaded: 72
  • Failed downloads: 1
  • Total word count: 351,883
  • Total data size: 2.06 MB
  • Average words per article: 4,887

📁 Data Location:
  • Articles: d:\My Projects\BizMind\premier_league_data\wikipedia
  • Metadata: d:\My Projects\BizMind\premier_league_data\metadata

📝 Content Categories:
  • General PL: 39 articles
  • Seasons: 1 articles
  • Clubs: 11 articles
  • Players: 7 articles
  • Managers: 5 articles
  • Other: 9 articles

✓ Data collection complete and ready for RAG training!


## Next Steps

The downloaded articles are now ready for RAG training:

1. **Load into Vector Database**: Use the articles in the `premier_league_data/wikipedia/` folder to populate your vector database (ChromaDB, as seen in your project).

2. **Create Embeddings**: Process the articles through your embedding model to create vector representations.

3. **Configure RAG**: Update your `rag_mind/rag_model.py` to point to this new data source.

4. **Test Queries**: Try asking questions about Premier League history to validate the RAG system.

Example questions to test:
- "Who won the Premier League in 1992-93?"
- "Tell me about the Invincibles season"
- "Who has scored the most Premier League goals?"
- "What was special about Leicester City in 2015-16?"